# Construcción dataset

En este apartado se pretende realizar la lectura de los ficheros y unificarlos en uno solo para el posterior entreno de la red, ya que ambos recogen datos de las mediciones de los sensores (acelerómetro, giroscopio y magnetómetro) por separado.

El dataset generado tendrá la información de todos los sensores formando un vector de 10 dimensiones, 9 dimensiones de entrada por los sensores más la última correspondiente a la salida.

En caso de que posteriormente se desee probar la red con menos entradas sería cuestión de saltar las columnas pertinentes.

In [1]:
import csv
import os
import random
import re
import shutil

datasetType = ["train","test"]
classification = {"parado":0,"caminar":1,"correr":2,"bici":3}
folder = "./data"
raw = "./rawdata"
headers = ["X(mg)","Y(mg)","Z(mg)","X(dps)","Y(dps)","Z(dps)","X(mGa)","Y(mGa)","Z(mGa)","Output"] #Acelerómetro,giroscopio y magnetómetro

+ ### Elección de parámetros
Para automatizar el proceso de la creación del fichero de entrenamiento, hay que identificar los parámetros necesarios para el correcto funcionamiento:

    + **Número de sensores.**
    + **Dato a clasificar.**
    + **Tipo de dataset (entrenamiento o validación).**
Este último parámetro lo establece el usuario indicandolo por teclado.

In [2]:
def initParams():
    try:
        for e in classification.keys():
            aux = "./" + e
            if not os.path.exists(aux):
                os.makedirs(aux)
            
        dataType = int(input("Especifique si los datos forman parte del conjunto de entrenamiento o validación: \n 0 --> Entrenamiento \n 1 --> Validación\n")) 
        if dataType > 1 or dataType < 0:
            raise Exception ("Valor entre 0 y 1, repita de nuevo")
        
        resultFolder = "./" + datasetType[dataType]
        if not os.path.exists(resultFolder):
            os.makedirs(resultFolder)

    except ValueError:
        print("Formato incorrecto")

    if not os.path.exists(folder):
        os.makedirs(folder)
        
    if not os.path.exists(raw):
        os.makedirs(raw)
    return dataType

 + ### Lectura de ficheros ###
 Para extraer las rutas de los ficheros creamos un diccionario en el que las claves son el tipo de clasificación al que pertenecen los datos y los valores las rutas de los ficheros.

In [3]:
def getRoutes():
    files = []
    rutas = [e for e in os.listdir() if e in classification.keys()]
    for e in rutas:
        for r in os.listdir(e):
            if re.search("\.csv",r):
                files.append("./" + e + "/" + r)
    files.sort()
    dicc = {i:[] for i in rutas}
    for r in rutas:
        for f in files:
            if r in f:
                dicc[r].append(f)
    return dicc

+ **A continuación almacenamos en una lista de listas todos los datos de los ficheros.**

In [4]:
def readFiles(files):
    dataFiles = []
    for f in files:
        aux = list()
        with open(f,'r') as file:
            lector = csv.reader(file,delimiter = ',')
            for i in range(4):
                next(lector)
            for e in lector:
                e = e[-4:-1]
                aux.append(e)
        file.close()
        dataFiles.append(aux)
    return dataFiles

+ **Ya solo queda formar un vector con los datos de los sensores alineados para introducirlo como entrada posteriormente a la red.**

In [5]:
def unifyFiles(dataFiles,output):
    unifiedData = []
    for z in range (0,len(dataFiles),3):
        for i in range (0,len(dataFiles[z])):
            aux = []
            for j in range(z,z+3):
                aux.extend(dataFiles[j][i])
            aux.append(output)
            unifiedData.append(aux)
    return unifiedData

+ **Con el vector de entrada alineado lo guardamos en un fichero.**
    
    Antes de completar el dataset almacenamos en ficheros por separado cada categoría a clasificar.

In [6]:
def writeFilescategory(unifiedData,output,fileName):
    ruta = folder + "/" + fileName + "/{}.csv".format(output)
    file = open(ruta, 'w',newline ='')
    #random.shuffle(unifiedData) #Comentar para comprobar que se guardan bien los datos
    with file:
        writer = csv.writer(file)
        writer.writerow(headers)
        writer.writerows(unifiedData)
    file.close()

+ **Por último generamos el dataset que emplearemos para entrenar a la red.**

In [7]:
def completeDataset(dataType,fileName):
    dataFiles = []
    directory = folder + "/" + fileName
    for e in os.listdir(directory):
         if re.search("\.csv",e):
            dataFiles.append(e)
   
    finalDataset = datasetType[dataType] + "/" + fileName + ".csv"
    aux = []
    for e in dataFiles:
        e = directory + "/" + e
        with open(e,'r') as t:
            reader = csv.reader(t,delimiter = ',')
            next(reader)
            for r in reader:
                aux.append(r)
        t.close()
    

    with open(finalDataset,'w',newline = '') as f:
        writer = csv.writer(f)
        writer.writerow(headers)
        writer.writerows(aux)
    f.close()

+ **Por último para completar el dataset solo quedaría llamar de forma ordenada las funciones definidas anteriormente.**

In [14]:
dataType = initParams()
fileName = input("Introduzca el nombre del fichero para guardar el dataset: ") 

datafolder = folder + "/" + fileName
if not os.path.exists(datafolder):
    os.makedirs(datafolder)

routes = getRoutes()
for k,v in routes.items():
    files = v
    output = classification[k]
    dataFiles = readFiles(files)
    uniFiedData = unifyFiles(dataFiles,output)
    writeFilescategory(uniFiedData,k,fileName)

completeDataset(dataType,fileName) #Realmente este fichero no se usa

Especifique si los datos forman parte del conjunto de entrenamiento o validación: 
 0 --> Entrenamiento 
 1 --> Validación
 1
Introduzca el nombre del fichero para guardar el dataset:  tfgtest


In [15]:
ls = [item for lista in routes.values() for item in lista]
moveFolder = raw + "/" + fileName
if not os.path.exists(moveFolder):
    os.makedirs(moveFolder)

for k,v in routes.items():
    r = moveFolder + "/" + k
    if not os.path.exists(r):
        os.makedirs(r)
    for e in v:
        shutil.move(e,r)